In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler


# Load the dataset
data = pd.read_csv("Bank Customer Churn Prediction.csv")

# Display the first few rows and summary info
data.head(30)